In [1]:
import pandas as pd
from math import log
import operator
from collections import Counter

In [2]:
data = pd.read_csv('../decision_tree_data.csv')
data

,hair,voice,label,height
0,long,thick,male,1.7
1,short,thick,male,1.7
2,short,thick,male,1.7
3,long,thin,female,1.6
4,short,thin,female,1.6
5,short,thick,female,1.6
6,long,thick,female,1.6
7,long,thick,female,1.7


# Define Functions Calculating Entropy and Gini

In [3]:
def calc_entropy(data):  
    num_entries = len(data) 
    label_counts = data['label'].value_counts()
    ent=0
    for key in label_counts.keys():
        prob = float(label_counts[key])/num_entries 
        ent -= prob*log(prob,2) # sum up all the entropy
    return ent

def calc_gini(data):
    num_entries = len(data)  
    label_counts = data['label'].value_counts()
    gini = 1
    for key in label_counts.keys():
        prob = float(label_counts[key])/num_entries 
        gini -= prob*prob 
    return gini

# Define Splitting Data Function

In [4]:
def splitdata(data, feature, value): # split by one value of one feature
    qualified_data = data[data[feature] == value]
    return qualified_data

# Define Diffenrent Ways to Identify the Best Features

In [5]:
# split based on information gain -- ID3
def split_ig(data):  
    base_entropy = calc_entropy(data)  
    best_ig = 0
    best_feature = -1
    
    features = [feature for feature in data.columns if 'label' != feature]
    for feature in features:
        unique_values = data[feature].unique()
        new_ent = 0
        for value in unique_values:
            subdata = splitdata(data,feature,value)
            prob =len(subdata)/float(len(data))
            new_ent += prob*calc_entropy(subdata)  
        info_gain = base_entropy - new_ent  
        if (info_gain>best_ig):   # choose the best feature to split
            best_ig=info_gain
            best_feature = feature
            
    return best_feature

#split based on information gain -- C4.5
def split_igratio(data): 
    base_entropy = calc_entropy(data) 
    best_ig_ratio = 0
    best_feature = -1
    
    features = [feature for feature in data.columns if 'label' != feature]
    for feature in features:
        unique_values = data[feature].unique()
        new_ent = 0
        for value in unique_values:
            subdata = splitdata(data,feature,value)
            prob =len(subdata)/float(len(data))
            new_ent += prob*calc_entropy(subdata)  
        info_gain_ratio = (base_entropy - new_ent)/new_ent 
        if (info_gain_ratio>best_ig_ratio):  
            best_ig_ratio=info_gain_ratio
            best_feature = feature
            
    return best_feature

#split based on Gini -- Cart
def split_gini(data):
    bestgini = 1
    best_feature = -1
    
    features = [feature for feature in data.columns if 'label' != feature]
    for feature in features:
        unique_values = data[feature].unique()
        new_gini = 0
        for value in unique_values:
            subdata = splitdata(data,feature,value)
            prob = len(subdata)/float(len(data))
            new_gini += prob*calc_gini(subdata)  
        if (new_gini < bestgini):  
            bestgini = new_gini
            best_feature = feature
            
    return best_feature

# Define Majority Vote

In [6]:
def majority_vote(data):    
    class_count = data['label'].value_counts()
    a = sorted(class_count.items(),key = lambda x:x[1],reverse = True)
    return a[0][0]

# Define Create Tree and Classify Function

In [7]:
def create_tree(data, split_method = 'ig'): # split_method -- default is information gain
    
    #if no label
    if 'label' not in data.columns:
        print('No label provided for training')
        return 'no label'
    
    #if only one class, return that class
    if len(data['label'].unique()) == 1:
        return data['label'].unique()[0]
    
    #if no feature data or feature data is not able to be split, return majority of the class
    if len(data.columns) == 1 or len(data[[feature for feature in data.columns if feature != 'label']].drop_duplicates()) == 1:
        return majority_vote(data)
    
    if split_method == 'gini':
        best_feature = split_gini(data) 
    elif split_method == 'igratio':
        best_feature = split_igratio(data)
    else:
        best_feature = split_ig(data)
        
    left_feature = [feature for feature in data.columns if feature != best_feature]
    important_features.append(best_feature)
    #find possible values for current best feature
    bestFeatLabel = data[best_feature].unique()
    
    #create tree
    myTree={best_feature:{}}

    for value in bestFeatLabel:
        current_data = splitdata(data, best_feature, value)[left_feature]
        myTree[best_feature][value]=create_tree(current_data)
        
    return myTree

In [8]:
def classify(data, tree, important_features, predicted_column):
    data[predicted_column] = ''
    for id in data.index:
        cur_tree = tree
        for feature in important_features:
            value = cur_tree[feature][data[feature][id]]
            if type(value) is str:
                data[predicted_column][id] = value
                break
            else:
                cur_tree = value
    return data

# Example

In [9]:
#create a list to note down features based on the order of importance
important_features = []
tree = create_tree(data)

In [10]:
classify(data, tree, important_features, 'igratio_predicted')

/Users/kehanliu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,hair,voice,label,height,igratio_predicted
0,long,thick,male,1.7,male
1,short,thick,male,1.7,male
2,short,thick,male,1.7,male
3,long,thin,female,1.6,female
4,short,thin,female,1.6,female
5,short,thick,female,1.6,female
6,long,thick,female,1.6,female
7,long,thick,female,1.7,male
